<h2><center> Лабораторна робота №4 </center></h2>
<h2><center> ДЕТЕКЦІЯ ОБЄКТІВ. </center></h2>
<h2><center> З дисципліни Обробка зображень методами штучного інтелекту </center></h2>

__Виконав:__
студент групи КН-414
Оренчук Олег

__Викладач:__ Пелешко Д.Д.

## Завдання
### Варіант №11
* Реалізувати детекцію об’єктів за допомогою мережі YoloV1.
* Провести порівняльний аналіз.


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the YOLOv1 network
net = cv2.dnn.readNetFromDarknet('drive/MyDrive/CV/yolo.cfg', 'drive/MyDrive/CV/yolo.weights')

# Load the input image
image = cv2.imread('drive/MyDrive/CV/bicycle.jpg')

# Preprocess the input image
blob = cv2.dnn.blobFromImage(image, 1/255.0, (448, 448), swapRB=True, crop=False)

# Set the network inputs
net.setInput(blob)

# Define the confidence threshold and non-maxima suppression threshold
confidence_thresh = 0.5
nms_thresh = 0.3

# Get the number of classes
num_classes = len(net.getLayerNames())

# Define the output layers
output_layers = net.getUnconnectedOutLayersNames()

# Forward pass through the network
outputs = net.forward(output_layers)

# Get the bounding boxes and class probabilities
boxes = []
confidences = []
class_ids = []

for output in outputs:
    for detection in output:
        # Get the class probabilities
        scores = detection[5:]

        # Get the class ID with the highest probability
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        # Filter out weak predictions
        if confidence > confidence_thresh:
            # Get the bounding box coordinates
            box = detection[0:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
            (centerX, centerY, width, height) = box.astype("int")

            # Calculate the top-left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            # Add the bounding box and class information to the lists
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_thresh, nms_thresh)

# Define the colors for each class label
colors = np.random.uniform(0, 255, size=(len(class_ids), 3))

# Draw the bounding boxes and class labels on the input image
if len(indices) > 0:
    for i in indices.flatten():
        # Get the coordinates of the bounding box
        x, y = boxes[i][0], boxes[i][1]
        w, h = boxes[i][2], boxes[i][3]

        # Draw the bounding box
        color = [int(c) for c in colors[i]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

        # Draw the class label and confidence score
        label = f"{net.getLayerNames()[class_ids[i]].title()} {confidences[i]:.2f}"
        cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Show the output image
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()